# Übung 6.1 - k-Means Clustering 
VO Maschinelles Lernen in der Produktion, WS2020/21, Moritz von Unold, Richard Lux, Felix Soest

#### In diesem Notebook wird das Verfahren k-Means Clustering anhand eines Anwendungsbeispiels geübt.

Im Gegensatz zur Regression und Klassifikation besteht der Datensatz für eine Clusteranalyse __nur aus Input-Parametern__. Ziel ist es den Datensatz in Klassen (Cluster) zu unterteilen.

Mittels des k-Means Clusterings soll der gegebene Datensatz in möglichst homogene Klassen (Cluster) unterteilt werden. Jeder Datenpunkt (Sample) wird hierbei einer Klasse (einem Cluster) zugeordnet.
 
### Data-Mining-Prozess:

![Bild konnte nicht geladen werden! 1. Daten erfassen - 2. Daten erkunden - 3. Daten vorbereiten - 4. Modelle bilden - 5. Modelle validieren - 6. Modell testen](Prozess_Modellentwicklung_v2.png "Title")

### 0. Bibliotheken importieren

In [ ]:
# Importiere benötigte Bibliotheken
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

pd.options.mode.chained_assignment = None
%matplotlib inline

### 1. Daten erfassen - Daten importieren

In [ ]:
# Erstelle eigene Zufallszahlen
my_seed = TODO

# Lade Datensatz
df = pd.read_excel("Daten_Clustering.xlsx")

### 2. Daten erkunden

In [ ]:
TODO

In [ ]:
TODO

In [ ]:
TODO

### 3. Daten vorbereiten

In [ ]:
# Aufteilen des Datensatzes in X und y - Nicht nötig da 'Clustering'
X_train = df.copy(deep=True)

### 4.1 Modell bilden - Mögliche Hyperparameter anzeigen

In [ ]:
# Importieren des Modells
from sklearn.cluster import KMeans

# Ausgabe möglicher Hyperparameter
print("\n Mögliche Hyperparameter (mit Standardeinstellungen) des k-Means Clustering:")
KMeans().get_params()

Beschreibung der Hyperparameter:
https://scikit-learn.org/stable/modules/generated/sklearn.cluster.KMeans.html

### 4.2 Modell bilden - Modell bilden

In [ ]:
# Modell bilden
kmeans = KMeans(n_clusters=TODO, init='k-means++', n_init=10, random_state=my_seed)

### 4.3 Modell bilden - Modell trainieren

In [ ]:
# Trainiere das Modell
kmeans = kmeans.fit(X_train)

In [ ]:
# Ausgabe Clusterzentren
kmeans.cluster_centers_

### 6.1 Modell testen & anwenden - Bewertung mittels Abstand zum Clusterzentrum

Um die Qualität/Güte des Modells zu bestimmen wird der Abstand eines jeden Samples zu seinem zugehörigen Clusterzentrum berechnet. Die Summe aller Abstände je Cluster und deren Summe lässt eine Schluss auf die Güte des Clusterings zu.

In [ ]:
# Mittlerer quadratischer Abstand aller Punkte zum Clusterzentrum
print("\ncluster:\t\tMean squared distance:\t\tnumber of points:")
mean_squared_distances = []
for i, cluster in enumerate(kmeans.cluster_centers_):
    mask = kmeans.labels_ == i
    mean_squared_distance = (
        (X_train["x1"][mask] - kmeans.cluster_centers_[i][0]) ** 2
        + (X_train["x2"][mask] - kmeans.cluster_centers_[i][1]) ** 2
        + (X_train["x3"][mask] - kmeans.cluster_centers_[i][2]) ** 2
    ).mean()
    mean_squared_distances.append(mean_squared_distance)
    print(
        "cluster "
        + str(i)
        + "\t\t"
        + str("%.2f" % mean_squared_distance)
        + "\t\t\t\t"
        + str(len(X_train["x1"][mask]))
    )
print(
    "mean (clusters)\t\t"
    + str("%.2f" % (sum(mean_squared_distances) / len(mean_squared_distances)))
    + "\t\t\t\t"
    + str(len(X_train))
)
%matplotlib inline
plt.figure(figsize=(16, 5))
plt.bar(range(len(kmeans.cluster_centers_)), mean_squared_distances)
plt.xlabel("cluster")
plt.ylabel("mean squared distance")
plt.show()

### 6.2 Modell testen & anwenden - Bewertung mittels Silhouette-Wert/-Form
Der Wert variiert zwischen -1 und 1. Ein Wert von 1 bedeutet das der Punkt einen großen Abstand zu den umliegenden anderen Clustern hat. Ein Wert von 0 bedeutet das der Punkt auf der Grenze zwischen 2 Clustern liegt. Ein Wert von -1 bedeutet das der Punkt wahr. dem falschen Cluster zuegordnet wurde.

In [ ]:
# Berechne Silhouetten-Wert
from sklearn.metrics import silhouette_samples, silhouette_score

print("mean silhouette:\t" + str("%.2f" % silhouette_score(X_train, kmeans.labels_)))

In [ ]:
# Ausgabe Silhouetten-Wert je Punkt je Cluster
df["silhouette"] = silhouette_samples(X_train, kmeans.labels_)
df["labels"] = kmeans.labels_
df = df.sort_values(["labels", "silhouette"])
%matplotlib inline
plt.figure(figsize=(16, 6))
plt.plot(range(len(df)), df["silhouette"])
plt.plot(
    [0, len(df)],
    [
        silhouette_score(X_train, kmeans.labels_),
        silhouette_score(X_train, kmeans.labels_),
    ],
)
plt.xlabel("every data point")
plt.ylabel("silhouette value")
plt.show()

### 6.3 Modell testen & anwenden - Visualisierung Clusterzentren

Um die Modellgüte visuell darzustellen wird ein 3D-Plot erstellt.

In [ ]:
# 3D-Plot Ergebnis Clustering
from mpl_toolkits.mplot3d import Axes3D

%matplotlib notebook

fig = plt.figure(figsize=(12, 12))
ax = fig.add_subplot(111, projection="3d")
legend = []

for i, cluster in enumerate(kmeans.cluster_centers_):
    if i == 0:
        color = "red"
    elif i == 1:
        color = "blue"
    elif i == 2:
        color = "green"
    elif i == 3:
        color = "yellow"
    elif i == 4:
        color = "brown"
    elif i == 5:
        color = "gray"
    elif i == 6:
        color = "olive"
    elif i == 7:
        color = "cyan"
    else:
        print("Warning - Too many clusters! -> Colours don't match any more!")
    mask = kmeans.labels_ == i
    ax.scatter(
        X_train["x1"][mask],
        X_train["x2"][mask],
        X_train["x3"][mask],
        c=color,
        marker="o",
        s=140,
    )
    ax.scatter(
        kmeans.cluster_centers_[i][0],
        kmeans.cluster_centers_[i][1],
        kmeans.cluster_centers_[i][2],
        c=color,
        marker="o",
        edgecolor="black",
        s=500,
    )
    legend.append("data points " + str(i))
    legend.append("cluster center " + str(i))

ax.set_xlabel("x1")
ax.set_ylabel("x2")
ax.set_zlabel("x3")
plt.legend(legend)
plt.show()

### Verständnisfragen:

1. Was unterscheidet Clusterverfahren von Regressions- und Klassifikationsverfahren?
2. Wie kann man ein Clustering-Ergebnis bewerten? 
3. Welche Clusteranzahl erscheint für dieses Problem sinnvoll, wie groß ist hier die Summe der mittleren quadratischen Abstände und der Silhouetten-Wert?